In [5]:
!pip install logomaker

In [46]:
import tensorflow as tf
import numpy as np

import h5py, os, io
import requests as rq

from tfomics import moana, evaluate, impress
import matplotlib.pyplot as plt

In [47]:
data = rq.get('https://www.dropbox.com/s/c3umbo5y13sqcfp/synthetic_dataset.h5?raw=true')
data.raise_for_status()

with h5py.File(io.BytesIO(data.content), 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32).transpose([0, 2, 1])
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32).transpose([0, 2, 1])
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32).transpose([0, 2, 1])
    y_test = np.array(dataset['Y_test']).astype(np.int32)

In [51]:
categories = [i for i in os.walk('results')][0][1]
for i in range(len(categories)):
    models = [i for i in os.walk(os.path.join('results', categories[i]))][0][1]
    for j in range(1, len(models)):
        num_filters = moana.count_meme_entries(os.path.join('motifs', categories[i], models[j] + '.meme'))
        motif_dir = os.path.join('results', categories[i], models[j], 'tomtom.tsv')
        
        match_frac, match_any, filter_matches, filter_qvalues, true_qvalues, true_hits = evaluate.motif_comparison_synthetic_dataset(motif_dir, num_filters=num_filters)
        
        """model = tf.keras.models.load_model(h5py.File(os.path.join('models', categories[i], models[j] + '.h5')))
        ppms = moana.filter_activations(x_test, model, layer=3, window=20, threshold=0.5)
        fig = plt.figure(figsize=(25, 4))
        impress.plot_filters(ppms, fig, num_cols=8, names=filter_matches, fontsize=14)"""